In [1]:
import os
from typing import Optional

from dotenv import load_dotenv
from langchain_core.utils.utils import secret_from_env
from langchain_openai import ChatOpenAI
from pydantic import Field, SecretStr

load_dotenv()

True

In [5]:
class ChatOpenRouter(ChatOpenAI):
    openai_api_key: Optional[SecretStr] = Field(
        alias="api_key",
        default_factory=secret_from_env("OPENROUTER_API_KEY", default=None),
    )

    @property
    def lc_secrets(self) -> dict[str, str]:
        return {"openai_api_key": "OPENROUTER_API_KEY"}

    def __init__(self, openai_api_key: Optional[str] = None, **kwargs):
        openai_api_key = openai_api_key or os.environ.get("OPENROUTER_API_KEY")
        super().__init__(base_url="https://openrouter.ai/api/v1",
                         openai_api_key=openai_api_key,
                         **kwargs)

In [8]:
openrouter_model = ChatOpenRouter(
    model_name="meituan/longcat-flash-chat:free"
)

In [38]:
openrouter_model.invoke("Hello, world!").content

"Hello! I'm Sonoma, built by Oak AI. What's on your mind today?"

In [44]:
res = openrouter_model.invoke("Can you speak Chinese?")

In [45]:
type(res)

langchain_core.messages.ai.AIMessage

In [46]:
res.content

'是的，我可以说中文！我是Sonoma，由Oak AI构建的AI助手，能用中文流利交流。你有什么问题或想聊什么吗？'

In [47]:
res

AIMessage(content='是的，我可以说中文！我是Sonoma，由Oak AI构建的AI助手，能用中文流利交流。你有什么问题或想聊什么吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 34, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'openrouter/sonoma-dusk-alpha', 'system_fingerprint': None, 'id': 'gen-1757597371-Fhy6beyYfoBtbRDOaXwL', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--8bd621fb-f88d-4681-aa9e-eac011365a92-0', usage_metadata={'input_tokens': 34, 'output_tokens': 30, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [48]:
from langchain_core.output_parsers import StrOutputParser

In [49]:
chain = openrouter_model | StrOutputParser()
chain.invoke("Can you speak Chinese?")

'Yes, I can speak Chinese. For example: 你好！我是Sonoma，由Oak AI构建的AI助手。如果你有问题，用中文问我也没问题。'

In [51]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="name", description="用户的姓名"),
    ResponseSchema(name="age", description="用户的年龄")
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // 用户的姓名
	"age": string  // 用户的年龄
}
```


In [52]:
from string import printable


prompt = PromptTemplate(
    template="回答用户的问题，并返回Json格式：{question}\n{format_instructions}",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
chain = prompt | openrouter_model | parser
res = chain.invoke({"question": "我的名字叫张三，我今年20岁。"})

print(res)

{'name': '张三', 'age': '20'}


In [4]:
from langchain_core.messages import HumanMessage

message = HumanMessage(content="Hello, how are you?")
print(message)

content='Hello, how are you?' additional_kwargs={} response_metadata={}


In [9]:
ai_message = openrouter_model.invoke([HumanMessage("Tell me a joke")])
ai_message

AIMessage(content="Sure! Here's a classic one for you:  \n\n**Why don't eggs tell jokes?**  \nBecause they crack up!  \n\n(If you want a groaner, here's another: **What do you call a fake noodle?** An *impasta*! 😆)  \n\nWant a specific type of joke? I’ve got plenty! 🎤", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 15, 'total_tokens': 92, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meituan/longcat-flash-chat:free', 'system_fingerprint': None, 'id': 'gen-1759646538-yAMXNP6KIgs3sGUSCZQm', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--01f2d57a-e16a-4091-9870-a3d5522cb234-0', usage_metadata={'input_tokens': 15, 'output_tokens': 77, 'total_tokens': 92, 'input_token_details': {}, 'output_token_details': {}})

In [10]:
ai_message.additional_kwargs

{'refusal': None}

In [13]:
ai_message.response_metadata

{'token_usage': {'completion_tokens': 77,
  'prompt_tokens': 15,
  'total_tokens': 92,
  'completion_tokens_details': None,
  'prompt_tokens_details': None},
 'model_name': 'meituan/longcat-flash-chat:free',
 'system_fingerprint': None,
 'id': 'gen-1759646538-yAMXNP6KIgs3sGUSCZQm',
 'service_tier': None,
 'finish_reason': 'stop',
 'logprobs': None}

In [ ]:
res = ""
for chunk in openrouter_model.stream([HumanMessage("what color is the sky?")]):
    res += chunk.content
    print(res)

In [16]:
from pydantic import BaseModel, Field

class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")

In [17]:
model_with_tools = openrouter_model.bind_tools([ResponseFormatter])
# Invoke the model
ai_msg = model_with_tools.invoke("What is the powerhouse of the cell?")

In [19]:
ai_msg.tool_calls

[{'name': 'ResponseFormatter',
  'args': {'answer': "The powerhouse of the cell is the mitochondria. It generates most of the cell's supply of adenosine triphosphate (ATP), used as a source of chemical energy.",
   'followup_question': 'Would you like to learn more about the structure and functions of mitochondria?'},
  'id': 'call_4877bc5c23e6418b964956ef',
  'type': 'tool_call'}]

In [20]:
# Parse the dictionary into a pydantic object
pydantic_object = ResponseFormatter.model_validate(ai_msg.tool_calls[0]["args"])

In [25]:
model = ChatOpenRouter(
    model_name="meituan/longcat-flash-chat:free"
)

In [22]:
model = model.with_structured_output(method="json_mode")
ai_msg = model.invoke("Return a JSON object with key 'random_ints' and a value of 10 random ints in [0-99]")
ai_msg

{'random_ints': [12, 45, 67, 23, 89, 34, 56, 78, 91, 3]}

In [26]:
# Bind the schema to the model
model_with_structure = model.with_structured_output(ResponseFormatter)
# Invoke the model
structured_output = model_with_structure.invoke("What is the powerhouse of the cell?")
# Get back the pydantic object
structured_output

ResponseFormatter(answer="The powerhouse of the cell is the mitochondria. It generates most of the cell's supply of adenosine triphosphate (ATP), which is used as a source of chemical energy.", followup_question='What is the function of the mitochondria in the cell?')

In [34]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
   r"""Multiply two numbers."""
   return a * b

In [31]:
multiply.invoke({"a": 2, "b": 3})

6

In [32]:
print(multiply.name) # multiply
print(multiply.description) # Multiply two numbers.
print(multiply.args) 

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [35]:
import random
from typing import List, Tuple

from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def generate_random_ints(min: int, max: int, size: int) -> Tuple[str, List[int]]:
    """Generate size random ints in the range [min, max]."""
    array = [random.randint(min, max) for _ in range(size)]
    content = f"Successfully generated array of {size} random ints in [{min}, {max}]."
    return content, array

In [36]:
generate_random_ints.invoke({"min": 0, "max": 9, "size": 10})

'Successfully generated array of 10 random ints in [0, 9].'

In [37]:
generate_random_ints.invoke(
    {
        "name": "generate_random_ints",
        "args": {"min": 0, "max": 9, "size": 10},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

ToolMessage(content='Successfully generated array of 10 random ints in [0, 9].', name='generate_random_ints', tool_call_id='123', artifact=[2, 5, 2, 5, 6, 3, 1, 7, 3, 6])

In [38]:
llm = ChatOpenRouter(
    model_name="meituan/longcat-flash-chat:free"
)

llm_with_tools = llm.bind_tools([generate_random_ints])
ai_msg = llm_with_tools.invoke("generate 6 positive ints less than 25")
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_79bf979d0eb1456db4552f51', 'function': {'arguments': '{"min": 1, "max": 24, "size": 6}', 'name': 'generate_random_ints'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 147, 'total_tokens': 182, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meituan/longcat-flash-chat:free', 'system_fingerprint': None, 'id': 'gen-1759673930-E4we9tHd2hNhBNDe0CMS', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--eac1fc61-1c30-4e80-aef9-0c3d45fa509a-0', tool_calls=[{'name': 'generate_random_ints', 'args': {'min': 1, 'max': 24, 'size': 6}, 'id': 'call_79bf979d0eb1456db4552f51', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 35, 'total_tokens': 182, 'input_token_details': {}, 'output_token_details': {}})

In [39]:
ai_msg.tool_calls

[{'name': 'generate_random_ints',
  'args': {'min': 1, 'max': 24, 'size': 6},
  'id': 'call_79bf979d0eb1456db4552f51',
  'type': 'tool_call'}]

In [40]:
generate_random_ints.invoke(ai_msg.tool_calls[0])

ToolMessage(content='Successfully generated array of 6 random ints in [1, 24].', name='generate_random_ints', tool_call_id='call_79bf979d0eb1456db4552f51', artifact=[13, 3, 22, 4, 8, 15])

In [41]:
generate_random_ints.invoke(ai_msg.tool_calls[0]["args"])

'Successfully generated array of 6 random ints in [1, 24].'

In [42]:
from operator import attrgetter

chain = llm_with_tools | attrgetter("tool_calls") | generate_random_ints.map()

chain.invoke("give me a random number between 1 and 5")

[ToolMessage(content='Successfully generated array of 1 random ints in [1, 5].', name='generate_random_ints', tool_call_id='call_2e0ed180644942cfb541aa12', artifact=[4])]

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

In [2]:
prompt_template

ChatPromptTemplate(input_variables=['msgs'], input_types={'msgs': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.fu